<a href="https://colab.research.google.com/github/gitHubAndyLee2020/OpenAI_Gym_RL_Algorithms_Database/blob/main/A2C_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### A2C

> About

- Given some state, the A2C model will output the probability of taking each possible action, and the expected reward for the state
- The A2C model is applied to multiple environments with different starting state to collect more data in parallel
- The difference between the actual reward from taking the selected action and the expected reward is used to calculate the loss value for each environment, and the mean of the loss is used to train the A2C model. This trains the neural network to maximize the probability of action that produces the maximum reward
- Same concept as Actor-Critic, except the Actor-Critic network is applied to multiple environments simultaneously.

> Pro

- Parallelism; more data collected simultaneously

> Con

- Complexity

```
Assume multiprocessing_env module is implemented already
```

```
def make_env():
  def _thunk():
    - Create environment and return it
  - Return the function to create environment
```

```
# Before model definition
- Initialize multiple environments
- Convert them into multi-processed environments
- Define test environment
```

```
class ActorCritic(nn.Module):
  def __init__(self, num_inputs, num_outputs, hidden_size, std=0.0):
    - Define critic network that maps inputs -> hidden layer -> estimated reward
    - Define actor network that maps inputs -> hidden layer -> action probabilities

  def forward(self, x):
    - Feed the input through the critic network and actor network, and convert the action probabilities to categorical distribution (ex: P(A) = 0.2, P(B) = 0.3, etc...)
    - Return the estimated reward and categorical action probabilities
```

```
def test_env(vis=False):
  - Test the trained model on the test environment
  - Run the model on the environment with the actions generated by the Actor-Critic network until the game is done
  - Return the total reward
```

```
def compute_returns(next_value, rewards, masks, gamma=0.99):
  - Compute R = r0 + gamma * r1 + gamma^2 * r2 + ... for all rewards, apply mask to ignore rewards after the agent is done. The next_value is the last reward value
  - Return the computed return values
```

```
# Run the following training loop and post training loop until the frame count of the agent playing the environment exceeds the max frame limit
# Miltiple environment is used during data collection to speed up the process
# Pre Training Loop
- Reset the multiple environments, each environment will have slightly different starting state
# Main Training Loop
- For x number of steps, feed the state of each environment to the Actor-Critic network, and select the action from the action categorical probability
- Apply the each action to its environment, and get the resulting reward for each environment
- Store the reward, action log probability, entropy, potential reward, and mask for each environment. Mask is a list of 0s and 1s where its 0 at "done", and 1 otherwise. This acts to ignore rewards after the agent is done in the environment. Entropy is the measure of uncertainty and randomness in the distribution
- Every y frames, run the test_env function above to test the performance of the current Actor-Critic network
# Post Traning Loop
- Continuing from the last state from each of the environment from the training loop, feed it to the Actor-Critic network and get the expected reward values; named "next_value" in the code
- Use the "next_value", collected rewards, and masks to calculate the temporarily discounted return value from compute_returns function from above
- Calculate the difference between the discounted returns values and collected expected values to get the advantage for each environment, which represents how well each environment performed with the current Actor compared to the expected return that the critic expected
- Actor loss is calculated by the mean of each negated log probability multiplied by each advantage. Higher advantage results in higher loss, which means the optimizer will adjust the neural network more towards the direction of producing higher probability for high-reward actions
- Critic loss is calculated by taking the mean of the square of the advantages. This represents the Mean Squared Error of the Critic network, where the network will try to minimize this value
- Actor loss and Critic loss is combined, and subtract by entropy. Higher advantage leads to both higher Actor loss and Critic loss, which works for both Actor network and Critic network (Actor network wants to move more towards favoring high-reward actions, Critic network wants to adjust more for more wrong predictions).
- Entropy is subtracted from the combined loss, because (1) exploration is encouraged. (2) High entropy means more uncertainty in the model choosing the actions. (2) Higher entropy decreases the loss value to maintain the current status-quo of exploratory nature
- Apply backpropagation
```

```
# Why does higher advantage leads to higher loss, and why does higher loss lead to the model favoring thoses actions?
# A higher positive advantage makes the loss term more positive.
Since the optimizer aims to minimize this loss, it will adjust the parameters such that log p(a∣s) becomes less negative, effectively making p(a∣s) larger. This means the probability of taking that action a in state s will increase, making it more likely to be chosen in the future.
```

In [ ]:
# Importing necessary modules
import math
import random
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
import matplotlib.pyplot as plt

# Check if CUDA is available and set the device accordingly
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Importing SubprocVecEnv for parallelizing environments
# from multiprocessing_env import SubprocVecEnv

# Number of parallel environments and name of the environment
num_envs = 8
env_name = "CartPole-v0"

# Function to create a new environment
def make_env():
    def _thunk():
        env = gym.make(env_name)
        return env
    return _thunk

# Enabling interactive mode for plotting
plt.ion()

# Create multiple environments for parallel execution
envs = [make_env() for i in range(num_envs)]
envs = SubprocVecEnv(envs)  # Create 8 parallel environments

# Create a single environment for testing
env = gym.make(env_name)

# Define the ActorCritic neural network model
class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size, std=0.0):
        super(ActorCritic, self).__init__()

        # Define the critic network
        self.critic = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )

        # Define the actor network
        self.actor = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs),
            nn.Softmax(dim=1),
        )

    # Forward pass for both actor and critic
    def forward(self, x):
        value = self.critic(x)
        probs = self.actor(x)
        dist = Categorical(probs)
        return dist, value

# Function to test the trained model in a single environment
def test_env(vis=False):
    state = env.reset()
    if vis: env.render()
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        dist, _ = model(state)
        next_state, reward, done, _ = env.step(dist.sample().cpu().numpy()[0])
        state = next_state
        if vis: env.render()
        total_reward += reward
    return total_reward

# Function to compute the discounted returns
def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

# Function to plot rewards during training
def plot(frame_idx, rewards):
    plt.plot(rewards, 'b-')
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.pause(0.0001)

# Define model's input and output dimensions
num_inputs = envs.observation_space.shape[0]
num_outputs = envs.action_space.n

# Hyperparameters
hidden_size = 256
lr = 1e-3
num_steps = 5

# Initialize the ActorCritic model and optimizer
model = ActorCritic(num_inputs, num_outputs, hidden_size).to(device)
optimizer = optim.Adam(model.parameters())

# Training setup
max_frames = 20000
frame_idx = 0
test_rewards = []

# Initial state reset
state = envs.reset()

# Main training loop
while frame_idx < max_frames:
    # Initialize lists to store trajectories
    log_probs = []
    values = []
    rewards = []
    masks = []
    entropy = 0

    # Rollout trajectory for a few steps
    for _ in range(num_steps):

        # Convert the state to a PyTorch float tensor and move it to the configured device (CPU or GPU).
        state = torch.FloatTensor(state).to(device)

        # Get the action distribution ('dist') and state value ('value') from the Actor-Critic model.
        dist, value = model(state)

        # Sample an action from the distribution.
        action = dist.sample()

        # Take the action in the environment and receive the next state, reward, and 'done' flag.
        next_state, reward, done, _ = envs.step(action.cpu().numpy())

        # Calculate the log probability of the taken action.
        log_prob = dist.log_prob(action)

        # Compute the entropy for the action distribution to encourage exploration.
        entropy += dist.entropy().mean()

        # Append the log probability, state value, reward, and mask to their respective lists.
        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))

        # Create a mask for the 'done' flag and append it to the masks list.
        # The mask will be 0 if the episode is done, otherwise 1.
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))

        # Update the current state to be the next state.
        state = next_state

        # Increment the frame index.
        frame_idx += 1

        # Test and plot the performance every 100 frames.
        if frame_idx % 100 == 0:

            # Evaluate the model's performance in the environment and take the mean over 10 runs.
            test_rewards.append(np.mean([test_env() for _ in range(10)]))

            # Plot the accumulated test rewards.
            plot(frame_idx, test_rewards)

    # Compute discounted returns
    next_state = torch.FloatTensor(next_state).to(device)
    _, next_value = model(next_state)
    returns = compute_returns(next_value, rewards, masks)

    # Convert lists to PyTorch tensors
    log_probs = torch.cat(log_probs)
    returns = torch.cat(returns).detach()
    values = torch.cat(values)

    # Calculate advantage and losses
    advantage = returns - values
    actor_loss = -(log_probs * advantage.detach()).mean()
    critic_loss = advantage.pow(2).mean()

    # Calculate total loss and perform backpropagation
    loss = actor_loss + 0.5 * critic_loss - 0.001 * entropy
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

#test_env(True)